In [ ]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

"""
This notebook just does everything in ./sh/preprocess.sh script
in an interactive manner.
"""

import sys
import os

sys.path.append('../')
sys.path.append('../paretoviz')
from paretoviz.utils import fmt
from paretoviz.utils import vectorops as vops

from paretoviz import normalizer as nrm
from paretoviz import tradeoff_calculator as tro
from paretoviz import peeler as plr
from paretoviz import palettizer as pltz

# Do all these plots with centroid based coloring

# rawfpath = "../data/spherical/spherical-3d.out"
# rawfpath = "../data/spherical/spherical-4d.out"
# rawfpath = "../data/spherical/spherical-8d.out"

# rawfpath = "../data/knee/knee-3d.out"
# rawfpath = "../data/knee/knee-4d.out"
# rawfpath = "../data/knee/knee-8d.out"

# rawfpath = "../data/line-surface/line-3d.out"
# rawfpath = "../data/line-surface/line-4d.out"
# rawfpath = "../data/line-surface/line-6d.out" # fails
# rawfpath = "../data/line-surface/line-8d.out"

# rawfpath = "../data/carcrash/carcrash-3d.out"
# rawfpath = "../data/carcrash/carcrash-c1/carcrash-c1-3d.out"
# rawfpath = "../data/carcrash/carcrash-c2/carcrash-c2-3d.out"

# Do all these plots with constraint based coloring

# rawfpath = "../data/knee-const/knee-const-3d.out"
# rawfpath = "../data/knee-const/knee-const-4d.out"
# rawfpath = "../data/knee-const/knee-const-8d.out"

# rawfpath = "../data/knee-const-mod/knee-const-mod-3d.out"
rawfpath = "../data/knee-const-mod/knee-const-mod-4d.out"

# rawfpath = "../data/isolated/isolated-3d.out"
# rawfpath = "../data/isolated/isolated-4d.out"
# rawfpath = "../data/isolated/isolated-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-4d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-5d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-c1/c2dtlz2-c1-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c2/c2dtlz2-c2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c3/c2dtlz2-c3-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c4/c2dtlz2-c4-3d.out"

# rawfpath = "../data/gaa-das/gaa-das-10d.out"
# rawfpath = "../data/gaa-lhs/gaa-lhs-10d.out"

# some ML data sets
# rawfpath = "../data/airofoil/airofoil-5d.out"
# rawfpath = "../data/banknote/banknote-4d.out"
# rawfpath = "../data/blood/blood-4d.out" # fails
# rawfpath = "../data/cccp/cccp-4d.out"
# rawfpath = "../data/concrete/concrete-8d.out" # fails
# rawfpath = "../data/iris/iris-4d.out" 
# rawfpath = "../data/mammogram/mammogram-5d.out" # fails
# rawfpath = "../data/wil/wil-7d.out" # fails
# rawfpath = "../data/yeast/yeast-8d.out" #  fails 

path, rawfname = os.path.split(rawfpath)
normfname = rawfname.split('.')[0] + "-norm.out"
mufname = normfname.split('.')[0] + "-mu.out"

In [ ]:
# This part does the normalizer.py operation

vals = fmt.load(rawfpath)

print("Normalizing {0:d} data points.".format(len(vals)))
vals_ = nrm.normalize(vals)
fmt.cat(vals_)

path, rawfile = os.path.split(rawfpath)
normfile = os.path.join(path, rawfile.split('.')[0] + "-norm.out")
print("Saving normalized data into {0:s} ...".format(normfile))
fmt.save(vals_, normfile)

In [ ]:
# This part does the tradeoff_calculator.py operation

# epsilon = 0.33 # spherical
# epsilon = 0.25 # knee
# epsilon = 0.025 # line
# epsilon = 0.25 # carcrash
epsilon = 0.125 # knee-const, mod
# epsilon = 0.33 # isolated
# epsilon = 0.33 # c2dtlz2-3d, 4d
# epsilon = 0.15 # c2dtlz2-5d

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

mu = tro.compute_tradeoff(points, epsilon, normalize = False)
fmt.cat(mu)

path, normfname = os.path.split(normfpath)
mufpath = os.path.join(path, normfname.split('.')[0] + "-mu.out")
print("Saving tradeoff values to {0:s} ...".format(mufpath))
fmt.save(mu, mufpath)

In [ ]:
# This part does the peeler.py operation

mode = "default"
# mode = "no-project"

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

m = len(points[0])
print("Peeling data point cloud in {0:s} mode ...".format(mode))
if mode == "default":
    # the defualt mode will project the points 
    # on a simplex on the first quadrant
    ppoints = plr.project(points)
    cpoints = plr.collapse(ppoints, dim = m - 1)
    boundaries = plr.peel(cpoints)
elif mode == "no-project":
    boundaries = plr.peel(points)
fmt.cat(boundaries, dtype = 'int')

layerfpath = os.path.join(path, normfname.split('.')[0] + "-layers.out")
print("Saving {0:d} layers into {1:s} ...".format(len(boundaries), layerfpath))
fmt.save(boundaries, layerfpath, dtype = 'int')

Peeling data point cloud in default mode ...
Total points: 3112
Computing alpha-shape ...


In [206]:
# This part does the palettizer.py operation

n_layers = 4
mode = "default"
mode = "star"
mode = "stardecay"

# tester()
normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

layerfile = normfname.split('.')[0] + "-layers.out"
layers = fmt.load(os.path.join(path, layerfile), dtype = 'int')

print("Computing palette coordinates with {0:d} layers in {1:s} mode ...".format(n_layers, mode))
if mode == "default":
    palette_coords = pltz.palettize(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette.out")
elif mode == "polar":
    palette_coords = pltz.palettize_polar(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-polar.out")
elif mode == "logistic":
    palette_coords = pltz.palettize_logistic(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-logistic.out")
elif mode == "star":
    palette_coords = pltz.palettize_star(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-star.out")
elif mode == "stardecay":
    palette_coords = pltz.palettize_stardecay(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-stardecay.out")
else:
    print("Error: unknown mode \'{0:s}\'\n".format(mode))
    sys.exit(1)

print("Saving palette coordinates into {0:s} ...".format(palettefpath))
pltz.save_palette(palette_coords, palettefpath)

Computing palette coordinates with 4 layers in stardecay mode ...
Spreading with e^(-4.00x)
Saving palette coordinates into ../data/gaa-lhs/gaa-lhs-10d-norm-palette-stardecay.out ...
